# GIFT Harmonic-Yukawa Pipeline v2.4

**Key improvement: Proper Yukawa from H²(21) + H³(77)**

$$Y_{ijk} = \int_{K_7} \omega_i \wedge \omega_j \wedge \Phi_k$$

where:
- $\omega_i, \omega_j$ are harmonic **2-forms** (b₂ = 21)
- $\Phi_k$ are harmonic **3-forms** (b₃ = 77)
- 2 + 2 + 3 = 7 → volume form on K₇ ✓

**Mass hierarchy** emerges from eigenvalues of $M_{ij} = \sum_k Y_{ijk}v_k$

In [ ]:
# @title Setup
!pip install torch numpy matplotlib --quiet

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Tuple, List, Optional
from itertools import combinations
from functools import lru_cache
import math, json

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")
torch.manual_seed(42)

In [ ]:
# @title GIFT v2.2 Constants
@dataclass
class Config:
    b2: int = 21                   # dim H²(K₇) - 2-forms
    b3: int = 77                   # dim H³(K₇) - 3-forms
    dim_2form: int = 21            # C(7,2)
    dim_3form: int = 35            # C(7,3)
    dim_4form: int = 35            # C(7,4)
    det_g_target: float = 65/32    # = 2.03125
    kappa_T: float = 1/61          # Torsion scale
    tau: float = 3472/891          # Hierarchy parameter
    v_higgs: float = 246.0         # GeV
    
    # TCS split
    n_visible: int = 43            # Visible sector
    n_hidden: int = 34             # Hidden sector (77-43)

cfg = Config()
print(f"b₂ = {cfg.b2}, b₃ = {cfg.b3}")
print(f"κ_T = {cfg.kappa_T:.6f}")
print(f"τ = {cfg.tau:.6f}")
print(f"Visible/Hidden: {cfg.n_visible}/{cfg.n_hidden}")

---
## Part 1: SPD Metric Network

In [ ]:
# @title Metric Network (Cholesky SPD)

class MetricNetwork(nn.Module):
    """g = L @ L.T via Cholesky parametrization."""
    
    def __init__(self, hidden=128, layers=4):
        super().__init__()
        self.B = nn.Parameter(torch.randn(7, 32) * 2, requires_grad=False)
        
        net = []
        d = 7 + 64
        for _ in range(layers-1):
            net += [nn.Linear(d, hidden), nn.SiLU()]
            d = hidden
        net.append(nn.Linear(hidden, 28))
        self.net = nn.Sequential(*net)
        
        # Init for det(g) ~ 65/32
        with torch.no_grad():
            bias = torch.zeros(28)
            bias[:7] = (65/32) ** (1/14)
            self.net[-1].bias.copy_(bias)
            self.net[-1].weight.mul_(0.01)
    
    def forward(self, x):
        proj = 2 * math.pi * x @ self.B
        feat = torch.cat([x, torch.sin(proj), torch.cos(proj)], -1)
        out = self.net(feat)
        
        L = torch.zeros(x.shape[0], 7, 7, device=x.device)
        L[:, range(7), range(7)] = F.softplus(out[:, :7]) + 0.1
        
        idx = 7
        for i in range(1, 7):
            for j in range(i):
                L[:, i, j] = out[:, idx] * 0.1
                idx += 1
        
        return L @ L.transpose(-1, -2)

metric_net = MetricNetwork().to(device)
print(f"Metric params: {sum(p.numel() for p in metric_net.parameters()):,}")

In [ ]:
# @title Train Metric

def train_metric(net, epochs=2000):
    opt = torch.optim.Adam(net.parameters(), lr=1e-3)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, epochs)
    
    for ep in range(epochs):
        opt.zero_grad()
        x = torch.rand(512, 7, device=device)
        g = net(x)
        det = torch.det(g)
        loss = ((det.mean() - cfg.det_g_target)**2) + 0.1 * det.var()
        loss.backward()
        opt.step()
        sched.step()
        
        if (ep+1) % 500 == 0:
            print(f"Ep {ep+1}: det={det.mean():.4f}±{det.std():.4f}")
    
    return net

print("Training metric...")
metric_net = train_metric(metric_net)

with torch.no_grad():
    det_val = torch.det(metric_net(torch.rand(1000,7,device=device)))
    print(f"\nFinal: det(g)={det_val.mean():.6f} (target {cfg.det_g_target:.6f})")

---
## Part 2: Exterior Calculus

Key operators:
- $d$: exterior derivative
- $*$: Hodge star (using metric g)
- $\delta = *d*$: codifferential
- $\wedge$: wedge product

In [ ]:
# @title Index Tables

@lru_cache(maxsize=1)
def idx2(): return list(combinations(range(7), 2))

@lru_cache(maxsize=1)
def idx3(): return list(combinations(range(7), 3))

@lru_cache(maxsize=1)
def idx4(): return list(combinations(range(7), 4))

# Precompute complementary indices for Hodge star
@lru_cache(maxsize=1)
def complement_2():
    """For each 2-form index, return (complement 5-form index, sign)."""
    i2 = idx2()
    comps = []
    for (i, j) in i2:
        comp = tuple(k for k in range(7) if k not in (i, j))
        full = (i, j) + comp
        inv = sum(1 for a in range(7) for b in range(a+1,7) if full[a] > full[b])
        comps.append((comp, (-1)**inv))
    return comps

@lru_cache(maxsize=1)
def complement_3():
    """For each 3-form index, return (complement 4-form index, sign)."""
    i3 = idx3()
    i4 = idx4()
    comps = []
    for (i, j, k) in i3:
        comp = tuple(m for m in range(7) if m not in (i, j, k))
        c4_idx = i4.index(comp)
        full = (i, j, k) + comp
        inv = sum(1 for a in range(7) for b in range(a+1,7) if full[a] > full[b])
        comps.append((c4_idx, (-1)**inv))
    return comps

print(f"2-form indices: {len(idx2())}")
print(f"3-form indices: {len(idx3())}")
print(f"4-form indices: {len(idx4())}")

In [ ]:
# @title Exterior Derivative for 2-forms and 3-forms

def exterior_d_2form(omega_fn, x, eps=1e-4):
    """Compute d(omega) for 2-form omega.
    d: Omega^2 -> Omega^3
    Returns: 3-form with C(7,3)=35 components
    """
    batch = x.shape[0]
    d_omega = torch.zeros(batch, 35, device=x.device)
    
    i2 = idx2()
    i3 = idx3()
    
    for c3, (i, j, k) in enumerate(i3):
        indices = [i, j, k]
        for pos, m in enumerate(indices):
            two = tuple(idx for idx in indices if idx != m)
            if two in i2:
                c2 = i2.index(two)
                sign = (-1) ** pos
                
                x_p = x.clone(); x_p[:, m] += eps
                x_m = x.clone(); x_m[:, m] -= eps
                
                omega_p = omega_fn(x_p)[:, c2]
                omega_m = omega_fn(x_m)[:, c2]
                
                d_omega[:, c3] += sign * (omega_p - omega_m) / (2*eps)
    
    return d_omega

def exterior_d_3form(omega_fn, x, eps=1e-4):
    """Compute d(omega) for 3-form omega.
    d: Omega^3 -> Omega^4
    Returns: 4-form with C(7,4)=35 components
    """
    batch = x.shape[0]
    d_omega = torch.zeros(batch, 35, device=x.device)
    
    i3 = idx3()
    i4 = idx4()
    
    for c4, (i, j, k, l) in enumerate(i4):
        indices = [i, j, k, l]
        for pos, m in enumerate(indices):
            three = tuple(idx for idx in indices if idx != m)
            if three in i3:
                c3 = i3.index(three)
                sign = (-1) ** pos
                
                x_p = x.clone(); x_p[:, m] += eps
                x_m = x.clone(); x_m[:, m] -= eps
                
                omega_p = omega_fn(x_p)[:, c3]
                omega_m = omega_fn(x_m)[:, c3]
                
                d_omega[:, c4] += sign * (omega_p - omega_m) / (2*eps)
    
    return d_omega

print("Exterior derivatives defined")

In [ ]:
# @title Hodge Star Operators

def hodge_star_2(omega, g):
    """Hodge star *: Ω² → Ω⁵."""
    batch = omega.shape[0]
    det_g = torch.det(g)
    vol = torch.sqrt(det_g.abs().clamp(min=1e-10))
    
    comps = complement_2()
    star_omega = torch.zeros(batch, 21, device=omega.device)  # 5-forms have 21 components
    
    for c2, ((comp, sign)) in enumerate(comps):
        star_omega[:, c2] = sign * vol * omega[:, c2]
    
    return star_omega

def hodge_star_3(omega, g):
    """Hodge star *: Ω³ → Ω⁴."""
    batch = omega.shape[0]
    det_g = torch.det(g)
    vol = torch.sqrt(det_g.abs().clamp(min=1e-10))
    
    i4 = idx4()
    comps = complement_3()
    star_omega = torch.zeros(batch, 35, device=omega.device)
    
    for c3, (c4_idx, sign) in enumerate(comps):
        star_omega[:, c4_idx] = sign * vol.unsqueeze(-1).expand(-1, 1).squeeze(-1) * omega[:, c3]
    
    return star_omega

print("Hodge star operators defined")

In [ ]:
# @title Codifferential via Gradient Norm

def codifferential_2form_norm(omega_fn, x, g, eps=1e-4):
    """||δω||² for 2-forms (approximated via grad of *ω)."""
    omega = omega_fn(x)
    delta_sq = torch.zeros(x.shape[0], device=x.device)
    
    for i in range(7):
        x_p = x.clone(); x_p[:, i] += eps
        x_m = x.clone(); x_m[:, i] -= eps
        
        star_p = hodge_star_2(omega_fn(x_p), g)
        star_m = hodge_star_2(omega_fn(x_m), g)
        
        d_star = (star_p - star_m) / (2*eps)
        delta_sq += (d_star ** 2).sum(dim=-1)
    
    return delta_sq

def codifferential_3form_norm(omega_fn, x, g, eps=1e-4):
    """||δω||² for 3-forms."""
    delta_sq = torch.zeros(x.shape[0], device=x.device)
    
    for i in range(7):
        x_p = x.clone(); x_p[:, i] += eps
        x_m = x.clone(); x_m[:, i] -= eps
        
        star_p = hodge_star_3(omega_fn(x_p), g)
        star_m = hodge_star_3(omega_fn(x_m), g)
        
        d_star = (star_p - star_m) / (2*eps)
        delta_sq += (d_star ** 2).sum(dim=-1)
    
    return delta_sq

print("Codifferential norms defined")

---
## Part 3: Harmonic Form Networks

Train networks for:
- **21 harmonic 2-forms** $\omega_i$ satisfying $d\omega = 0$, $\delta\omega = 0$
- **77 harmonic 3-forms** $\Phi_k$ satisfying $d\Phi = 0$, $\delta\Phi = 0$

In [ ]:
# @title Harmonic 2-Form Network

class Harmonic2FormNet(nn.Module):
    """Network outputting a harmonic 2-form ω(x)."""
    
    def __init__(self, hidden=64, layers=3, form_id=0):
        super().__init__()
        self.form_id = form_id
        
        torch.manual_seed(100 + form_id)
        self.B = nn.Parameter(torch.randn(7, 16) * 2, requires_grad=False)
        
        net = []
        d = 7 + 32
        for _ in range(layers-1):
            net += [nn.Linear(d, hidden), nn.Tanh()]
            d = hidden
        net.append(nn.Linear(hidden, 21))  # 2-form: 21 components
        self.net = nn.Sequential(*net)
        
        torch.manual_seed(200 + form_id)
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight, gain=0.5)
                nn.init.zeros_(m.bias)
    
    def forward(self, x):
        proj = 2 * math.pi * x @ self.B
        feat = torch.cat([x, torch.sin(proj), torch.cos(proj)], -1)
        return self.net(feat)

print("Harmonic 2-form network defined")

In [ ]:
# @title Harmonic 3-Form Network

class Harmonic3FormNet(nn.Module):
    """Network outputting a harmonic 3-form Φ(x)."""
    
    def __init__(self, hidden=64, layers=3, form_id=0):
        super().__init__()
        self.form_id = form_id
        
        torch.manual_seed(1000 + form_id)
        self.B = nn.Parameter(torch.randn(7, 16) * 2, requires_grad=False)
        
        net = []
        d = 7 + 32
        for _ in range(layers-1):
            net += [nn.Linear(d, hidden), nn.Tanh()]
            d = hidden
        net.append(nn.Linear(hidden, 35))  # 3-form: 35 components
        self.net = nn.Sequential(*net)
        
        torch.manual_seed(2000 + form_id)
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight, gain=0.5)
                nn.init.zeros_(m.bias)
    
    def forward(self, x):
        proj = 2 * math.pi * x @ self.B
        feat = torch.cat([x, torch.sin(proj), torch.cos(proj)], -1)
        return self.net(feat)

print("Harmonic 3-form network defined")

In [ ]:
# @title Training Functions

def train_harmonic_2form(form_id, metric_net, prev_nets, epochs=500, verbose=True):
    """Train a single harmonic 2-form."""
    net = Harmonic2FormNet(form_id=form_id).to(device)
    opt = torch.optim.Adam(net.parameters(), lr=5e-4)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, epochs)
    
    for ep in range(epochs):
        opt.zero_grad()
        
        x = torch.rand(256, 7, device=device)
        with torch.no_grad():
            g = metric_net(x)
        
        # Closedness: ||dω||² = 0
        d_omega = exterior_d_2form(net, x)
        loss_closed = (d_omega ** 2).sum(dim=-1).mean()
        
        # Coclosedness: ||δω||² = 0
        loss_coclosed = codifferential_2form_norm(net, x, g).mean()
        
        # Normalization
        omega = net(x)
        det_g = torch.det(g)
        vol = torch.sqrt(det_g.abs().clamp(min=1e-10))
        norm_sq = ((omega ** 2).sum(dim=-1) * vol).mean()
        loss_norm = (norm_sq - 1.0) ** 2
        
        # Orthogonality
        loss_ortho = torch.tensor(0.0, device=device)
        for prev in prev_nets:
            with torch.no_grad():
                omega_prev = prev(x)
            inner = ((omega * omega_prev).sum(dim=-1) * vol).mean()
            loss_ortho = loss_ortho + inner ** 2
        
        loss = 5*loss_closed + 5*loss_coclosed + 10*loss_norm + 20*loss_ortho
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), 1.0)
        opt.step()
        sched.step()
        
        if verbose and (ep+1) % 250 == 0:
            print(f"  Ep {ep+1}: L={loss.item():.4f} "
                  f"(d={loss_closed.item():.4f}, δ={loss_coclosed.item():.4f})")
    
    return net

def train_harmonic_3form(form_id, metric_net, prev_nets, epochs=500, verbose=True):
    """Train a single harmonic 3-form."""
    net = Harmonic3FormNet(form_id=form_id).to(device)
    opt = torch.optim.Adam(net.parameters(), lr=5e-4)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, epochs)
    
    for ep in range(epochs):
        opt.zero_grad()
        
        x = torch.rand(256, 7, device=device)
        with torch.no_grad():
            g = metric_net(x)
        
        # Closedness
        d_phi = exterior_d_3form(net, x)
        loss_closed = (d_phi ** 2).sum(dim=-1).mean()
        
        # Coclosedness
        loss_coclosed = codifferential_3form_norm(net, x, g).mean()
        
        # Normalization
        phi = net(x)
        det_g = torch.det(g)
        vol = torch.sqrt(det_g.abs().clamp(min=1e-10))
        norm_sq = ((phi ** 2).sum(dim=-1) * vol).mean()
        loss_norm = (norm_sq - 1.0) ** 2
        
        # Orthogonality
        loss_ortho = torch.tensor(0.0, device=device)
        for prev in prev_nets:
            with torch.no_grad():
                phi_prev = prev(x)
            inner = ((phi * phi_prev).sum(dim=-1) * vol).mean()
            loss_ortho = loss_ortho + inner ** 2
        
        loss = 5*loss_closed + 5*loss_coclosed + 10*loss_norm + 20*loss_ortho
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), 1.0)
        opt.step()
        sched.step()
        
        if verbose and (ep+1) % 250 == 0:
            print(f"  Ep {ep+1}: L={loss.item():.4f}")
    
    return net

print("Training functions defined")

In [ ]:
# @title Train Harmonic 2-Forms (b₂ = 21)

n_2forms = 21  # Full b₂
harmonic_2forms = []

print(f"Training {n_2forms} harmonic 2-forms...\n")

for i in range(n_2forms):
    print(f"2-Form {i+1}/{n_2forms}:")
    net = train_harmonic_2form(i, metric_net, harmonic_2forms, epochs=400, verbose=True)
    harmonic_2forms.append(net)
    print()

print(f"Trained {len(harmonic_2forms)} harmonic 2-forms")

In [ ]:
# @title Train Harmonic 3-Forms (subset of b₃ = 77)

n_3forms = 20  # Subset for demo (full would be 77)
harmonic_3forms = []

print(f"Training {n_3forms} harmonic 3-forms (demo subset of 77)...\n")

for i in range(n_3forms):
    print(f"3-Form {i+1}/{n_3forms}:")
    net = train_harmonic_3form(i, metric_net, harmonic_3forms, epochs=400, verbose=True)
    harmonic_3forms.append(net)
    print()

print(f"Trained {len(harmonic_3forms)} harmonic 3-forms")

---
## Part 4: Yukawa Tensor from Wedge Product

$$Y_{ijk} = \int_{K_7} \omega_i \wedge \omega_j \wedge \Phi_k$$

This is the proper topological coupling giving fermion masses.

In [ ]:
# @title Wedge Product Tables

def build_wedge_22_table():
    """Precompute 2∧2 → 4 structure."""
    i2 = idx2()
    i4 = idx4()
    table = []  # (idx_a, idx_b, idx_4, sign)
    
    for a, (i, j) in enumerate(i2):
        for b, (k, l) in enumerate(i2):
            indices = (i, j, k, l)
            if len(set(indices)) != 4:
                continue
            
            sorted_indices = tuple(sorted(indices))
            idx_4 = i4.index(sorted_indices)
            
            # Sign from permutation
            perm = sorted(range(4), key=lambda x: indices[x])
            inv = sum(1 for a in range(4) for b in range(a+1,4) if perm[a] > perm[b])
            sign = (-1) ** inv
            
            table.append((a, b, idx_4, sign))
    
    return table

def build_wedge_43_table():
    """Precompute 4∧3 → 7 structure."""
    i3 = idx3()
    i4 = idx4()
    table = []  # (idx_4, idx_3, sign)
    
    for c, (i, j, k) in enumerate(i3):
        for d, four_idx in enumerate(i4):
            all_7 = set(four_idx) | {i, j, k}
            if len(all_7) != 7:
                continue
            
            full = four_idx + (i, j, k)
            perm = sorted(range(7), key=lambda x: full[x])
            inv = sum(1 for a in range(7) for b in range(a+1,7) if perm[a] > perm[b])
            sign = (-1) ** inv
            
            table.append((d, c, sign))
    
    return table

wedge_22 = build_wedge_22_table()
wedge_43 = build_wedge_43_table()

print(f"2∧2→4 entries: {len(wedge_22)}")
print(f"4∧3→7 entries: {len(wedge_43)}")

In [ ]:
# @title Compute Yukawa Tensor Y_ijk

def compute_yukawa(omega_nets, phi_nets, metric_net, n_points=3000):
    """Compute Y_ijk = integral omega_i wedge omega_j wedge Phi_k.
    
    Args:
        omega_nets: List of 2-form networks (21)
        phi_nets: List of 3-form networks
        metric_net: Metric network for volume form
        n_points: Monte Carlo integration points
    
    Returns:
        Y: Yukawa tensor (21, 21, n_3forms)
    """
    n2 = len(omega_nets)
    n3 = len(phi_nets)
    
    Y = torch.zeros(n2, n2, n3, device=device)
    
    with torch.no_grad():
        x = torch.rand(n_points, 7, device=device)
        g = metric_net(x)
        det_g = torch.det(g)
        vol = torch.sqrt(det_g.abs().clamp(min=1e-10))
        
        # Evaluate all forms
        omega = torch.stack([net(x) for net in omega_nets], dim=1)  # (n_pts, n2, 21)
        phi = torch.stack([net(x) for net in phi_nets], dim=1)      # (n_pts, n3, 35)
        
        print(f"Computing Yukawa tensor ({n2}×{n2}×{n3})...")
        
        # Wedge product: omega_i ∧ omega_j → 4-form
        for i in range(n2):
            for j in range(n2):
                # Compute omega_i ∧ omega_j
                eta = torch.zeros(n_points, 35, device=device)
                
                for (a, b, c4, sign) in wedge_22:
                    eta[:, c4] += sign * omega[:, i, a] * omega[:, j, b]
                
                # Wedge with each 3-form: eta ∧ Phi_k → 7-form
                for k in range(n3):
                    scalar = torch.zeros(n_points, device=device)
                    
                    for (c4, c3, sign) in wedge_43:
                        scalar += sign * eta[:, c4] * phi[:, k, c3]
                    
                    # Integrate with volume form
                    Y[i, j, k] = (scalar * vol).mean()
            
            if (i+1) % 5 == 0:
                print(f"  {i+1}/{n2} rows")
    
    return Y

Y = compute_yukawa(harmonic_2forms, harmonic_3forms, metric_net)
print(f"\nYukawa tensor shape: {Y.shape}")
print(f"Yukawa range: [{Y.min():.4f}, {Y.max():.4f}]")

In [ ]:
# @title Mass Matrix from Yukawa

# Contract Yukawa with VEV-like weights
# M_ij = sum_k Y_ijk * v_k
# For now, use uniform VEV
v_k = torch.ones(len(harmonic_3forms), device=device)
v_k = v_k / v_k.norm()  # Normalize

M = torch.einsum('ijk,k->ij', Y, v_k)

# Mass eigenvalues from M^† M
mass_matrix = M @ M.T
mass_eigs, mass_vecs = torch.linalg.eigh(mass_matrix)
mass_eigs = mass_eigs.clamp(min=0)
idx = torch.argsort(mass_eigs, descending=True)
mass_eigs = mass_eigs[idx]

# Convert to masses with proper scaling
# m = v_Higgs * sqrt(eigenvalue) * kappa_T
masses = cfg.v_higgs * torch.sqrt(mass_eigs) * cfg.kappa_T

print("\nMass spectrum (with κ_T = 1/61 scaling):")
print(f"Masses (GeV): {masses[:9].cpu().numpy()}")

# Compute hierarchy ratios
if masses.min() > 1e-15:
    ratio_max_min = (masses[0] / masses[-1]).item()
    print(f"\nm_max/m_min = {ratio_max_min:.2f}")

In [ ]:
# @title Tau Computation

# tau = sum_visible / sum_hidden eigenvalues
# For demo with n_3forms < 77, we estimate

visible_sum = mass_eigs[:cfg.n_visible].sum()
# Scale hidden contribution based on ratio
if len(mass_eigs) > cfg.n_visible:
    hidden_sum = mass_eigs[cfg.n_visible:].sum() * (cfg.n_hidden / max(1, len(mass_eigs) - cfg.n_visible))
else:
    # Extrapolate hidden sector from lowest eigenvalues
    lowest = mass_eigs[-min(5, len(mass_eigs)):].mean()
    hidden_sum = lowest * cfg.n_hidden

if hidden_sum > 1e-15:
    tau_computed = (visible_sum / hidden_sum).item()
else:
    tau_computed = float('inf')

print(f"Computed τ = {tau_computed:.4f}")
print(f"Target τ = {cfg.tau:.4f}")
print(f"Error: {abs(tau_computed - cfg.tau)/cfg.tau * 100:.1f}%")

In [ ]:
# @title Visualization

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Yukawa tensor slice
ax = axes[0, 0]
Y_slice = Y[:, :, 0].cpu().numpy()  # First 3-form coupling
im = ax.imshow(Y_slice, cmap='RdBu', vmin=-Y_slice.max(), vmax=Y_slice.max())
ax.set_title('Yukawa Y_{ij,0}')
ax.set_xlabel('j (2-form)')
ax.set_ylabel('i (2-form)')
plt.colorbar(im, ax=ax)

# 2. Mass matrix
ax = axes[0, 1]
M_np = M.cpu().numpy()
im = ax.imshow(M_np, cmap='RdBu', vmin=-abs(M_np).max(), vmax=abs(M_np).max())
ax.set_title('Mass Matrix M_{ij}')
ax.set_xlabel('j')
ax.set_ylabel('i')
plt.colorbar(im, ax=ax)

# 3. Mass eigenvalue spectrum (log scale)
ax = axes[1, 0]
ax.semilogy(mass_eigs.cpu().numpy() + 1e-15, 'b.-', markersize=8)
ax.set_xlabel('Mode')
ax.set_ylabel('Eigenvalue')
ax.set_title('Mass² Eigenvalue Spectrum')
ax.grid(True, alpha=0.3)

# 4. Mass comparison with PDG
ax = axes[1, 1]
pdg_masses = [172.69, 4.18, 1.777, 1.27, 0.10566, 0.0934, 0.00467, 0.00216, 0.000511]
n_show = min(len(masses), len(pdg_masses))
ax.semilogy(range(n_show), masses[:n_show].cpu().numpy() + 1e-15, 'bo-', 
            label='Computed', markersize=8)
ax.semilogy(range(len(pdg_masses)), pdg_masses, 'r^-', 
            label='PDG', markersize=10)
ax.set_xlabel('Rank')
ax.set_ylabel('Mass (GeV)')
ax.set_title('Mass Hierarchy Comparison')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('harmonic_yukawa_v24.png', dpi=150)
plt.show()
print("Saved: harmonic_yukawa_v24.png")

In [ ]:
# @title Mass Ratios

# Key GIFT ratios
if len(masses) >= 9 and masses[8] > 1e-15:
    tau_e = (masses[2] / masses[8]).item() if masses[8] > 1e-15 else float('inf')
    print(f"m_τ/m_e = {tau_e:.1f} (GIFT: 3477)")

if len(masses) >= 8 and masses[7] > 1e-15:
    s_d = (masses[5] / masses[7]).item() if masses[7] > 1e-15 else float('inf')
    print(f"m_s/m_d = {s_d:.1f} (GIFT: 20)")

# Koide parameter
if len(masses) >= 3:
    sqrt_m = torch.sqrt(masses[:3])
    Q_koide = sqrt_m.sum()**2 / (3 * masses[:3].sum())
    print(f"Q_Koide = {Q_koide.item():.4f} (GIFT: 0.6667)")

In [ ]:
# @title Export Results

results = {
    "version": "2.4",
    "n_2forms": len(harmonic_2forms),
    "n_3forms": len(harmonic_3forms),
    "det_g": {
        "computed": det_g.mean().item() if 'det_g' in dir() else None,
        "target": cfg.det_g_target
    },
    "yukawa": {
        "shape": list(Y.shape),
        "max": Y.max().item(),
        "min": Y.min().item()
    },
    "mass_eigenvalues": mass_eigs.cpu().tolist(),
    "masses_GeV": masses.cpu().tolist(),
    "tau": {
        "computed": tau_computed,
        "target": cfg.tau
    },
    "scaling": {
        "v_higgs": cfg.v_higgs,
        "kappa_T": cfg.kappa_T
    }
}

with open('harmonic_yukawa_v24_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("Exported: harmonic_yukawa_v24_results.json")

---
## Summary

**v2.4 key improvements:**
1. **Proper Yukawa**: $Y_{ijk} = \int \omega_i \wedge \omega_j \wedge \Phi_k$ (2+2+3=7)
2. **Both form types**: H²(21) 2-forms + H³(77) 3-forms
3. **Wedge product tables**: Precomputed for efficient computation
4. **κ_T = 1/61 scaling**: Proper torsion normalization

**Expected outcome:**
- Eigenvalue hierarchy from topological couplings
- Mass ratios approaching PDG values
- τ = 3472/891 from visible/hidden split